In [2]:
from __future__ import absolute_import
from __future__ import division
from __future__ import print_function

import csv
import pickle5 as pickle
import string
import gzip

from gensim.models.word2vec import Word2Vec

DATA_DIR = '../data/'

In [3]:
regular_diagnosis = pickle.load(open( f'{DATA_DIR}regular_diagnosis.pickle', "rb" ) )
print('len regular_diagnosis: ', len(regular_diagnosis))

len regular_diagnosis:  14222


In [4]:
filters = '!"#$%&()*+,-./:;<=>?@[\]^_`{|}~\t\n'
tt = str.maketrans(string.digits + filters,
                      'd' * len(string.digits) + ' ' * len(filters))

with open(f'{DATA_DIR}infrequent_word_mapping.pickle', 'rb') as f:
    infrequent_word_mapping = pickle.load(f)

seqs = []
# with open('notes.csv', 'rb') as f:
with gzip.open(f'{DATA_DIR}NOTEEVENTS.csv.gz', 'rt') as f:
    reader = csv.reader(f)
    for row in reader:
        key = (row[1], row[2])
        if key in regular_diagnosis:
            tokens = row[-1].lower().translate(tt).strip().split()
            for i, word in enumerate(tokens):
                tokens[i] = infrequent_word_mapping.get(word, word)
            seqs.append(tokens)


In [6]:
print('Building word embeddings.')

model = Word2Vec(seqs, vector_size=300, workers=4)
model.wv.save_word2vec_format(f'{DATA_DIR}word2vec_embeddings.vec', binary=False)

Building word embeddings.


In [5]:
with open(f'{DATA_DIR}word2vec_embeddings.vec', 'rb') as f:
    embeddings_dim = int(next(f).split()[0])
    print(embeddings_dim)

53229
